### Converting Grosmark's data to neuropy format

In [ ]:
from neuropy.core import Neurons, Epoch
import numpy as np
import scipy.io as sio
from pathlib import Path
import pandas as pd

basepath = Path("/data/Clustering/sessions/GrosmarkReclusteredData/")

sessions = [
    "Achilles_10252013",
    "Achilles_11012013",
    "Buddy_06272013",
    "Cicero_09172014",
    "Gatsby_08282013",
]
spkdata = sio.loadmat(basepath / "CRCNSReclustered-spikes.mat")["spikes"]
statesdata = sio.loadmat(basepath / "CRCNSReclustered-behavior.mat")["behavior"]
# prdigmdata = basepath/'wake_new/wake-behavior.mat'

for i, name in enumerate(sessions):
    sessdata = spkdata[name][0][0]
    spiketrains = sessdata["time"].squeeze()
    neuron_type_id = sessdata["quality"].astype(float).squeeze()
    neuron_type = np.ones(len(neuron_type_id), dtype="U5")
    neuron_type[neuron_type_id < 4] = "pyr"
    # neuron_type[neuron_type_id == 6] = "mua"
    neuron_type[neuron_type_id == 8] = "inter"

    shank_ids = np.array([_.astype(int).squeeze() for _ in sessdata["id"].squeeze()])
    shank_ids = shank_ids[:, 0]

    sessbehav = statesdata[name][0][0]

    pre = sessbehav["PREEpoch"][0][0]
    maze = sessbehav["MazeEpoch"][0][0]
    post = sessbehav["POSTEpoch"][0][0]
    paradigm_epoch = np.vstack((pre, maze, post))

    wake_epoch = Epoch.from_array(*sessbehav["Wake"][0][0].T, "AW")
    nrem_epoch = Epoch.from_array(*sessbehav["NREM"][0][0].T, "NREM")
    qw_epoch = Epoch.from_array(*sessbehav["Drowsy"][0][0].T, "QW")
    im_epoch = Epoch.from_array(*sessbehav["Intermediate"][0][0].T, "QW")
    rem_epoch = Epoch.from_array(*sessbehav["REM"][0][0].T, "REM")

    neurons = Neurons(
        spiketrains=spiketrains,
        t_start=paradigm_epoch[0, 0],
        t_stop=paradigm_epoch[-1, -1],
        shank_ids=shank_ids,
        neuron_type=neuron_type,
    )

    paradigm = Epoch.from_array(
        paradigm_epoch[:, 0], paradigm_epoch[:, 1], ["pre", "maze", "post"]
    )

    brainstates = wake_epoch + nrem_epoch + qw_epoch + im_epoch + rem_epoch
    brainstates = brainstates.duration_slice(0.5)

    sesspath = basepath / name
    # sesspath.mkdir()
    neurons.save(sesspath / f"{name}.neurons.npy")
    paradigm.save(sesspath / f"{name}.paradigm.npy")
    brainstates.save(sesspath / f"{name}.brainstates.npy")


In [ ]:
brainstates.to_dataframe()
